In [ ]:
from main import *
from models import *
from tqdm import tqdm

beta_1 = 1e-4
beta_T = 0.02
T = 500
shape = (44, 188, 4)
device = torch.device('cuda')

model = DiffusionModel(device, beta_1, beta_T, T)
process = DiffusionProcess(beta_1, beta_T, T, model, device, shape)
optim = torch.optim.Adam(model.parameters(), lr = 1e-4)

total_iteration = 1500
current_iteration = 0

train_sub = [1]

train_x0, train_y0 = make_train_dataset(train_sub, 0)
print(train_x0[0].shape)
train_stft = Stft_dataset(train_x0, train_y0)

dataloader = torch.utils.data.DataLoader(train_stft, batch_size = 64, drop_last = True, num_workers = 0)

pbar = tqdm(range(total_iteration))


for epoch in pbar:
    losses = []
    num_items = 0
    
    for x, y in dataloader:
        data = x.to(device = device)
        y = y.to(device=device)
        loss = loss_fn(model, x, idx=None, y=y)

        optim.zero_grad()
        loss.backward()
        optim.step()
        
        losses.append(loss.item())
        num_items += x.shape[0]
    pbar.set_description("Average loss : {}".format(sum(losses) / num_items))

In [ ]:
only_final = True
process = DiffusionProcess(beta_1, beta_T, T, model, device, shape)
samples = sample(process, num_images=32)

In [ ]:
import matplotlib.pyplot as plt


vis = train_x0[0]

for stft in vis:
    plt.plot(stft)
    plt.show()
    break

sample0 = samples[0]

for stft in sample0:
    plt.plot(stft)
    plt.show()
    break

val_sub = [9]
test_sub = [9]
train_x, _, _, _, _, _ = make_classifier_dataset(train_sub, val_sub, test_sub)
vis = train_x[0]

for signal in vis:
    plt.plot(signal)
    plt.show()
    break
    
generated_signal = return_to_signal(samples)
for signal in generated_signal[0]:
    plt.plot(signal)
    plt.show()
    break
    


In [ ]:
from main import *

train_sub = [1]
val_sub = [2]
test_sub = [2]

device = 'cuda'
main(train_sub, val_sub, test_sub, device)


2023-05-17 20:22:01.023895: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-17 20:22:01.483772: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/junyeobe/.local/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
  0%|                                                                                                                                                                                                                                 | 0/1 [00:00<?, ?it/s]/home/junye

Generated signal shape :  (64, 22, 1126)
Generated signal shape :  (64, 22, 1126)
Generated signal shape :  (64, 22, 1126)


/home/junyeobe/.local/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:1492: UserWarning: NOLA condition failed, STFT may not be invertible
  warnings.warn("NOLA condition failed, STFT may not be invertible")


Generated signal shape :  (64, 22, 1126)
generated signal length :  256


2023-05-17 20:24:46.214310: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-17 20:24:46.248621: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-17 20:24:46.248907: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-17 20:24:46.250108: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-17 20:24:46.250360: I tensorflow/compile

Epoch 1/1000


In [ ]:
# 실험결과 
# 그냥 stft 바로 집어넣었을 때 loss 0.002 부근

# stft / 10 했을 때 loss 더 줄어든다
